# AMPSphere v.2022-03

This is a notebook meant to form the set of notebooks used to analyze the data in AMPSphere and write the manuscript:

__AMPSphere: Global survey of prokaryotic antimicrobial peptides shaping microbiomes__

### Analysis of abundance and diversity of c_AMPs

c_AMPs are distributed as gene variants through many species. Here, we will test:
    
    I. Is there multi-habitat c_AMPs?
    II. Are multi-habitat c_AMPs clonal?

In [1]:
import lzma
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from Bio import SeqIO
from tqdm import tqdm
from scipy.stats import norm
from scipy.stats import shapiro
from scipy.stats import pearsonr, spearmanr

In [2]:
higher_level = {'sediment' : 'other',
        'bird gut' : 'other animal',
        'cat gut' : 'non-human mammal gut',
        'insect associated' : 'other animal',
        'human urogenital tract' : 'other human',
        'dog gut' : 'non-human mammal gut',
        'fermented food' : 'anthropogenic',
        'groundwater' : 'aquatic',
        'coral associated' : 'other animal',
        'rat gut' : 'non-human mammal gut',
        'human associated' : 'other human',
        'cattle gut' : 'non-human mammal gut',
        'deer gut' : 'non-human mammal gut',
        'mouse gut' : 'non-human mammal gut',
        'river associated' : 'aquatic',
        'primate gut' : 'non-human mammal gut',
        'human respiratory tract' : 'other human',
        'cattle rumen' : 'other animal',
        'human saliva' : 'other human',
        'activated sludge' : 'anthropogenic',
        'lake associated' : 'aquatic',
        'wastewater' : 'anthropogenic',
        'chicken gut' : 'other animal',
        'air' : 'other',
        'human mouth' : 'other human',
        'plant associated' : 'soil/plant',
        'water associated' : 'aquatic',
        'pig gut' : 'non-human mammal gut',
        'human skin' : 'other human',
        'marine' : 'aquatic',
        'soil' : 'soil/plant',
        'built environment' : 'anthropogenic',
        'human gut' : 'human gut',
        'anthropogenic': 'anthropogenic',
        'bear gut' : 'non-human mammal gut',
        'bee gut': 'other animal',
        'bat gut': 'non-human mammal gut',
        'dog associated': 'other animal',
        'cattle associated': 'other animal',
        'crustacean associated': 'other animal',
        'insect gut': 'other animal',
        'goat gut': 'non-human mammal gut', 
        'rodent gut': 'non-human mammal gut',
        'fisher gut': 'non-human mammal gut',
        'human digestive tract': 'other human',
        'coyote gut': 'non-human mammal gut',
        'planarian associated': 'other animal',
        'sponge associated': 'other animal',
        'goat rumen': 'other animal',
        'crustacean gut': 'other animal',
        'annelidae associated': 'other animal',
        'bird skin': 'other animal',
        'beatle gut': 'other animal',
        'termite gut': 'other animal', 
        'fish gut': 'other animal',
        'mollusc associated': 'other animal',
        'ship worm associated': 'other animal',
        'rabbit gut': 'non-human mammal gut',
        'tunicate associated': 'other animal',
        'mussel associated': 'other animal',
        'horse gut': 'non-human mammal gut',
        'wasp gut': 'other animal',
        'guinea pig gut': 'non-human mammal gut'}

In [3]:
# load data
data = pd.read_table('data/gmsc_amp_genes_envohr_source.tsv.gz')

In [4]:
# eliminate genes/amps without environment (from ProGenomes)
data = data[~data.general_envo_name.isna()]

# attribut high-level habitat to genes/amps
data['high'] = data.general_envo_name.map(lambda x: higher_level.get(x, 'other'))

In [5]:
# testing multihabitat AMPs
def get_multihabitat(df, level=None):
    '''
    counts the number of multi-habitat AMPs (present in at least 3 environments)
    
    :inputs:
    data frame containing at least AMP, general_envo_name, high-level habitat
    level which stats for the habitat or high-level environment
    
    :outputs:
    length of the list of multi-habitat c_AMPs
    '''
    if level == None:
        level = 'high'
    if level == 'low':
        level = 'general_envo_name'
        
    h = df[['amp', level]].drop_duplicates()
    h = h.amp.value_counts()
    h = h[h > 2].index
    
    return h

In [6]:
l = get_multihabitat(data, 'high')
l0 = get_multihabitat(data, 'low')
print(f'Multi-habitat AMPs: {len(l0)}\nMulti-high-level-habitat: {len(l)}')

Multi-habitat AMPs: 26346
Multi-high-level-habitat: 10606


In [7]:
k = data[['amp', 'high']].drop_duplicates()['amp'].value_counts()
k = k[k>2].index

with open('multihabitat_highlevel.txt', 'wt') as out:
    for i in k: out.write(f'{i}\n')

k = data[['amp', 'general_envo_name']].drop_duplicates()['amp'].value_counts()
k = k[k>2].index

with open('multihabitat_generalenvo.txt', 'wt') as out:
    for i in k: out.write(f'{i}\n')

### Permutation test

We shuffle the high-level habitat annotation for the samples, and then, calculate the number of multi-habitat c_AMPs. This operation is repeated 100 times, and then we calculate the average and standard deviation of the distribution of random results. Using Shapiro-Wilk test, we check if the random distribution is normal, and if it is, we calculate the Z-score for the result obtained for AMPSphere. The Z-score is then converted into a p-value to support our conclusions.

In [8]:
# testing significance
def shuffle_test(df, level=None):
    if level == None: level = 'high'
    elif level == 'low': level = 'general_envo_name'
        
    habitat = df.set_index('sample')[level].to_dict()
    
    values = [v for _, v in habitat.items()]
    random.shuffle(values)
    
    for k, v in zip(habitat, values):
        habitat[k] = v
        
    altdf = df.copy()
    altdf[level] = altdf['sample'].map(lambda x: habitat.get(x))
    
    return altdf

In [9]:
#test high level habitats
test = []
for _ in tqdm(range(32)):
    altdf = shuffle_test(data, 'high')
    altdf = len(get_multihabitat(altdf, 'high'))
    test.append(altdf)

print(test)

100%|███████████████████████████████████████████| 32/32 [13:15<00:00, 24.87s/it]

[247452, 253865, 253870, 252060, 255273, 254350, 255768, 250557, 250331, 253964, 255878, 254275, 252594, 257028, 257803, 254158, 257213, 254708, 250760, 258274, 254739, 250537, 254988, 252776, 254090, 253171, 260193, 251705, 257012, 258343, 255871, 254805]


In [10]:
_, p = shapiro(test)

if p < 0.05: res = 'not-normal'
else: res = 'normal'

print(f'The Shapiro-Wilks test returned a p = {p}')
print(f'This means that the distribution is {res}')

avg, std = np.mean(test), np.std(test)

print(f'Average number of random multi-habitat AMPs - {avg}, with std = {std}')

Z = (len(l) - avg) / std

pz = norm.sf(abs(Z))

print(f'The number of multi-habitat AMPs in AMPSphere was {len(l)}')
print(f'It was {Z} * std of the random distribution')
print(f'This gives us a p-value of {pz}')

The Shapiro-Wilks test returned a p = 0.9087615609169006
This means that the distribution is normal
Average number of random multi-habitat AMPs - 254325.34375, with std = 2702.486871029337
The number of multi-habitat AMPs in AMPSphere was 10606
It was -90.18335902485659 * std of the random distribution
This gives us a p-value of 0.0


In [11]:
#test habitats
test_low = []
for _ in tqdm(range(32)):
    altdf = shuffle_test(data, 'low')
    altdf = len(get_multihabitat(altdf, 'low'))
    test_low.append(altdf)

print(test_low)

100%|███████████████████████████████████████████| 32/32 [13:20<00:00, 25.00s/it]

[264118, 267295, 270065, 273581, 270293, 268954, 272104, 269219, 271854, 269617, 270630, 264865, 268658, 271951, 268026, 275134, 272292, 267355, 270904, 263027, 268803, 271308, 272162, 268219, 271158, 270246, 271824, 271884, 268741, 268316, 270421, 273183]


In [12]:
_, p = shapiro(test_low)

if p < 0.05: res = 'not-normal'
else: res = 'normal'

print(f'The Shapiro-Wilks test returned a p = {p}')
print(f'This means that the distribution is {res}')

avg, std = np.mean(test_low), np.std(test_low)

print(f'Average number of random multi-habitat AMPs - {avg}, with std = {std}')

Z = (len(l0) - avg) / std

pz = norm.sf(abs(Z))

print(f'The number of multi-habitat AMPs in AMPSphere was {len(l0)}')
print(f'It was {Z} * std of the random distribution')
print(f'This gives us a p-value of {pz}')

The Shapiro-Wilks test returned a p = 0.23920124769210815
This means that the distribution is normal
Average number of random multi-habitat AMPs - 269881.46875, with std = 2636.3309374058936
The number of multi-habitat AMPs in AMPSphere was 26346
It was -92.37666838201842 * std of the random distribution
This gives us a p-value of 0.0


### Testing gene variants enrichment in multi-habitat c_AMPs

In [13]:
def seqrev(seq):
    '''
    Checks if (1) sequence starts with an inverse complement of stop codon
    and if (2) ends in a stop codon.
     
    If first is true and second is true - do nothing
    If first is true and second is false - do reverse complement
    If first is false - do nothing
    '''
    if (seq[0:3] in {'TTA', 'CTA', 'TCA'}):
        if (seq[-3:] in {'TAA', 'TAG', 'TGA'}):
            pass
        else:
            return seq.translate(str.maketrans('ACGT', 'TGCA'))[::-1]
    return seq


def enrichment(k, x, N, m):
    '''
    Enrichment test using hypergeometric distribution
    :inputs:
    k - number of differentially abundant proteins
    x - number of genes in the input list with the annotation
    N - background number of genes
    m - background number of genes with annotation
    '''
    from scipy.stats import hypergeom
    return hypergeom.sf(x, N, k, m)

In [14]:
ntseqs = 'data/AMPSphere_v.2022-03.fna.xz'

# collecting genes
hseqs = []
with lzma.open(ntseqs, 'rt') as infile:
    for record in SeqIO.parse(infile, 'fasta'):
        h = record.description.split(' | ')[1]
        s = seqrev(str(record.seq))
        hseqs.append((h, s))

# checking number of gene variants
hseqs = set(hseqs)
df = pd.DataFrame(hseqs, columns=['AMP', 'gene'])
df = df.sort_values(by='AMP')
vargenes = df.AMP.value_counts()

In [15]:
# general habitat -- multihabitat AMPs
x = vargenes.loc[l0]
m = len(vargenes[vargenes > 2])
N = len(vargenes)
k, x = len(x), len(x[x>2])
p = enrichment(k, x, N, m)
enr = (x/k) / (m/N)

print(f'''Enrichment of multi-variant AMPs among the multi-habitat -- general envo:
Proportion: {x*100/k}  Background proportion: {m*100/N}
Enrichment: {enr}        p-value: {p}''')

Enrichment of multi-variant AMPs among the multi-habitat -- general envo:
Proportion: 55.83769832232597  Background proportion: 10.52972907870082
Enrichment: 5.302861821513773        p-value: 0.0


In [16]:
# high-level habitats -- multihabitat AMPs
x = vargenes.loc[l]
m = len(vargenes[vargenes > 2])
N = len(vargenes)
k, x = len(x), len(x[x>2])
p = enrichment(k, x, N, m)
enr = (x/k) / (m/N)

print(f'''Enrichment of multi-variant AMPs among the multi-habitat -- high envo:
Proportion: {x*100/k}  Background proportion: {m*100/N}
Enrichment: {enr}        p-value: {p}''')

Enrichment of multi-variant AMPs among the multi-habitat -- high envo:
Proportion: 59.2966245521403  Background proportion: 10.52972907870082
Enrichment: 5.631353295887119        p-value: 0.0
